In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def firsts(list):
    answer=[]
    for pair in list:
        answer=answer+[pair[0]]
    return answer

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def last_element(listo):return listo[len(listo)-1]
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [3]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=5
for k in [1..10]: # k = 1 => power = 2
    data=[]
    power=s[k][0]
    print("----------------------------------------------------------------------------")
    print("power: ",power);print()
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    column=[]
    for m in [3..503]:
        mp=ZZ(mod(m,p))
        mp2=ZZ(mod(m,p^2))
        dsm=digitsum(p,m)
        mpm=ZZ(mod(power,m))
        ex=order(p,m) 
        core=m/p^ex
        cmp=ZZ(mod(core,p))
        polydm=evaluate(polyd,m)
        o=order(p,polydm)
        zmo2=ZZ(mod(o,2))
        data=data+[zmo2]
    print(data)

----------------------------------------------------------------------------
power:  2

[0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1,

[0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 

In [4]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=5
for k in [1..10]: # k = 1 => power = 2
    data=[]
    power=s[k][0]
    print("----------------------------------------------------------------------------")
    print("power: ",power);print()
    poly=D(s[k][1]) # poly(m) = cst term of j_m^power
    nf=QQ(numericalfactor(poly))
    polyd=expand(poly/nf)
    column=[]
    for m in [3..503]:
        mp=ZZ(mod(m,p))
        mp2=ZZ(mod(m,p^2))
        dsm=digitsum(p,m)
        mpm=ZZ(mod(power,m))
        ex=order(p,m) 
        core=m/p^ex
        cmp=ZZ(mod(core,p))
        polydm=evaluate(polyd,m)
        o=order(p,polydm)
        zmo2=ZZ(mod(o,2))
        if cmp-mp==1:data=data+[zmo2]
    print(data)

----------------------------------------------------------------------------
power:  2

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
----------------------------------------------------------------------------
power:  3

[1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
----------------------------------------------------------------------------
power:  4

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
----------------------------------------------------------------------------
power:  5

[1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
----------------------------------------------------------------------------
power:  6

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
----------------------------------------------------------------------------
power:  7

[1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
----------------

In [19]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [3..3]:
    p=prime(n)
    for k in [1..20]: 
        data=[]
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..200]:
            m=p^j
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            data=data+[o]
        if data==[]:print("power: ",power," prime: ",p," empty")
        if data!=[]:
            print("----------------------------------------------------------------------------------")
            print()
            ul=uniteList(data)
            print("power: ",power," prime: ",p)
            print(ul);print()
            dt=difference_table(data);print();print(dt)

----------------------------------------------------------------------------------

power:  2  prime:  5
[3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45, 47, 49, 51, 53, 55, 57, 59, 61, 63, 65, 67, 69, 71, 73, 75, 77, 79, 81, 83, 85, 87, 89, 91, 93, 95, 97, 99, 101, 103, 105, 107, 109, 111, 113, 115, 117, 119, 121, 123, 125, 127, 129, 131, 133, 135, 137, 139, 141, 143, 145, 147, 149, 151, 153, 155, 157, 159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183, 185, 187, 189, 191, 193, 195, 197, 199, 201, 203, 205, 207, 209, 211, 213, 215, 217, 219, 221, 223, 225, 227, 229, 231, 233, 235, 237, 239, 241, 243, 245, 247, 249, 251, 253, 255, 257, 259, 261, 263, 265, 267, 269, 271, 273, 275, 277, 279, 281, 283, 285, 287, 289, 291, 293, 295, 297, 299, 301, 303, 305, 307, 309, 311, 313, 315, 317, 319, 321, 323, 325, 327, 329, 331, 333, 335, 337, 339, 341, 343, 345, 347, 349, 351, 353, 355, 357, 359, 361, 363, 365, 367, 369, 371, 373, 375, 377, 379, 38

----------------------------------------------------------------------------------

power:  8  prime:  5
[5, 13, 21, 29, 37, 45, 53, 61, 69, 77, 85, 93, 101, 109, 117, 125, 133, 141, 149, 157, 165, 173, 181, 189, 197, 205, 213, 221, 229, 237, 245, 253, 261, 269, 277, 285, 293, 301, 309, 317, 325, 333, 341, 349, 357, 365, 373, 381, 389, 397, 405, 413, 421, 429, 437, 445, 453, 461, 469, 477, 485, 493, 501, 509, 517, 525, 533, 541, 549, 557, 565, 573, 581, 589, 597, 605, 613, 621, 629, 637, 645, 653, 661, 669, 677, 685, 693, 701, 709, 717, 725, 733, 741, 749, 757, 765, 773, 781, 789, 797, 805, 813, 821, 829, 837, 845, 853, 861, 869, 877, 885, 893, 901, 909, 917, 925, 933, 941, 949, 957, 965, 973, 981, 989, 997, 1005, 1013, 1021, 1029, 1037, 1045, 1053, 1061, 1069, 1077, 1085, 1093, 1101, 1109, 1117, 1125, 1133, 1141, 1149, 1157, 1165, 1173, 1181, 1189, 1197, 1205, 1213, 1221, 1229, 1237, 1245, 1253, 1261, 1269, 1277, 1285, 1293, 1301, 1309, 1317, 1325, 1333, 1341, 1349, 1357, 1365, 1373, 

----------------------------------------------------------------------------------

power:  13  prime:  5
[10, 23, 36, 49, 62, 75, 88, 101, 114, 127, 140, 153, 166, 179, 192, 205, 218, 231, 244, 257, 270, 283, 296, 309, 322, 335, 348, 361, 374, 387, 400, 413, 426, 439, 452, 465, 478, 491, 504, 517, 530, 543, 556, 569, 582, 595, 608, 621, 634, 647, 660, 673, 686, 699, 712, 725, 738, 751, 764, 777, 790, 803, 816, 829, 842, 855, 868, 881, 894, 907, 920, 933, 946, 959, 972, 985, 998, 1011, 1024, 1037, 1050, 1063, 1076, 1089, 1102, 1115, 1128, 1141, 1154, 1167, 1180, 1193, 1206, 1219, 1232, 1245, 1258, 1271, 1284, 1297, 1310, 1323, 1336, 1349, 1362, 1375, 1388, 1401, 1414, 1427, 1440, 1453, 1466, 1479, 1492, 1505, 1518, 1531, 1544, 1557, 1570, 1583, 1596, 1609, 1622, 1635, 1648, 1661, 1674, 1687, 1700, 1713, 1726, 1739, 1752, 1765, 1778, 1791, 1804, 1817, 1830, 1843, 1856, 1869, 1882, 1895, 1908, 1921, 1934, 1947, 1960, 1973, 1986, 1999, 2012, 2025, 2038, 2051, 2064, 2077, 2090, 2103, 2116,

----------------------------------------------------------------------------------

power:  18  prime:  5
[11, 29, 47, 65, 83, 101, 119, 137, 155, 173, 191, 209, 227, 245, 263, 281, 299, 317, 335, 353, 371, 389, 407, 425, 443, 461, 479, 497, 515, 533, 551, 569, 587, 605, 623, 641, 659, 677, 695, 713, 731, 749, 767, 785, 803, 821, 839, 857, 875, 893, 911, 929, 947, 965, 983, 1001, 1019, 1037, 1055, 1073, 1091, 1109, 1127, 1145, 1163, 1181, 1199, 1217, 1235, 1253, 1271, 1289, 1307, 1325, 1343, 1361, 1379, 1397, 1415, 1433, 1451, 1469, 1487, 1505, 1523, 1541, 1559, 1577, 1595, 1613, 1631, 1649, 1667, 1685, 1703, 1721, 1739, 1757, 1775, 1793, 1811, 1829, 1847, 1865, 1883, 1901, 1919, 1937, 1955, 1973, 1991, 2009, 2027, 2045, 2063, 2081, 2099, 2117, 2135, 2153, 2171, 2189, 2207, 2225, 2243, 2261, 2279, 2297, 2315, 2333, 2351, 2369, 2387, 2405, 2423, 2441, 2459, 2477, 2495, 2513, 2531, 2549, 2567, 2585, 2603, 2621, 2639, 2657, 2675, 2693, 2711, 2729, 2747, 2765, 2783, 2801, 2819, 2837, 2855,

In [20]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [4..4]:
    p=prime(n)
    for k in [1..20]: 
        data=[]
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..200]:
            m=p^j
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            data=data+[o]
        if data==[]:print("power: ",power," prime: ",p," empty")
        if data!=[]:
            print("----------------------------------------------------------------------------------")
            print()
            ul=uniteList(data)
            print("power: ",power," prime: ",p)
            print(ul);print()
            dt=difference_table(data);print();print(dt)

----------------------------------------------------------------------------------

power:  2  prime:  7
[2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198, 200, 202, 204, 206, 208, 210, 212, 214, 216, 218, 220, 222, 224, 226, 228, 230, 232, 234, 236, 238, 240, 242, 244, 246, 248, 250, 252, 254, 256, 258, 260, 262, 264, 266, 268, 270, 272, 274, 276, 278, 280, 282, 284, 286, 288, 290, 292, 294, 296, 298, 300, 302, 304, 306, 308, 310, 312, 314, 316, 318, 320, 322, 324, 326, 328, 330, 332, 334, 336, 338, 340, 342, 344, 346, 348, 350, 352, 354, 356, 358, 360, 362, 364, 366, 368, 370, 372, 374, 376, 378, 38

----------------------------------------------------------------------------------

power:  7  prime:  7
[5, 12, 19, 26, 33, 40, 47, 54, 61, 68, 75, 82, 89, 96, 103, 110, 117, 124, 131, 138, 145, 152, 159, 166, 173, 180, 187, 194, 201, 208, 215, 222, 229, 236, 243, 250, 257, 264, 271, 278, 285, 292, 299, 306, 313, 320, 327, 334, 341, 348, 355, 362, 369, 376, 383, 390, 397, 404, 411, 418, 425, 432, 439, 446, 453, 460, 467, 474, 481, 488, 495, 502, 509, 516, 523, 530, 537, 544, 551, 558, 565, 572, 579, 586, 593, 600, 607, 614, 621, 628, 635, 642, 649, 656, 663, 670, 677, 684, 691, 698, 705, 712, 719, 726, 733, 740, 747, 754, 761, 768, 775, 782, 789, 796, 803, 810, 817, 824, 831, 838, 845, 852, 859, 866, 873, 880, 887, 894, 901, 908, 915, 922, 929, 936, 943, 950, 957, 964, 971, 978, 985, 992, 999, 1006, 1013, 1020, 1027, 1034, 1041, 1048, 1055, 1062, 1069, 1076, 1083, 1090, 1097, 1104, 1111, 1118, 1125, 1132, 1139, 1146, 1153, 1160, 1167, 1174, 1181, 1188, 1195, 1202, 1209, 1216, 1223, 12

----------------------------------------------------------------------------------

power:  12  prime:  7
[10, 22, 34, 46, 58, 70, 82, 94, 106, 118, 130, 142, 154, 166, 178, 190, 202, 214, 226, 238, 250, 262, 274, 286, 298, 310, 322, 334, 346, 358, 370, 382, 394, 406, 418, 430, 442, 454, 466, 478, 490, 502, 514, 526, 538, 550, 562, 574, 586, 598, 610, 622, 634, 646, 658, 670, 682, 694, 706, 718, 730, 742, 754, 766, 778, 790, 802, 814, 826, 838, 850, 862, 874, 886, 898, 910, 922, 934, 946, 958, 970, 982, 994, 1006, 1018, 1030, 1042, 1054, 1066, 1078, 1090, 1102, 1114, 1126, 1138, 1150, 1162, 1174, 1186, 1198, 1210, 1222, 1234, 1246, 1258, 1270, 1282, 1294, 1306, 1318, 1330, 1342, 1354, 1366, 1378, 1390, 1402, 1414, 1426, 1438, 1450, 1462, 1474, 1486, 1498, 1510, 1522, 1534, 1546, 1558, 1570, 1582, 1594, 1606, 1618, 1630, 1642, 1654, 1666, 1678, 1690, 1702, 1714, 1726, 1738, 1750, 1762, 1774, 1786, 1798, 1810, 1822, 1834, 1846, 1858, 1870, 1882, 1894, 1906, 1918, 1930, 1942, 1954, 1966, 

----------------------------------------------------------------------------------

power:  17  prime:  7
[13, 30, 47, 64, 81, 98, 115, 132, 149, 166, 183, 200, 217, 234, 251, 268, 285, 302, 319, 336, 353, 370, 387, 404, 421, 438, 455, 472, 489, 506, 523, 540, 557, 574, 591, 608, 625, 642, 659, 676, 693, 710, 727, 744, 761, 778, 795, 812, 829, 846, 863, 880, 897, 914, 931, 948, 965, 982, 999, 1016, 1033, 1050, 1067, 1084, 1101, 1118, 1135, 1152, 1169, 1186, 1203, 1220, 1237, 1254, 1271, 1288, 1305, 1322, 1339, 1356, 1373, 1390, 1407, 1424, 1441, 1458, 1475, 1492, 1509, 1526, 1543, 1560, 1577, 1594, 1611, 1628, 1645, 1662, 1679, 1696, 1713, 1730, 1747, 1764, 1781, 1798, 1815, 1832, 1849, 1866, 1883, 1900, 1917, 1934, 1951, 1968, 1985, 2002, 2019, 2036, 2053, 2070, 2087, 2104, 2121, 2138, 2155, 2172, 2189, 2206, 2223, 2240, 2257, 2274, 2291, 2308, 2325, 2342, 2359, 2376, 2393, 2410, 2427, 2444, 2461, 2478, 2495, 2512, 2529, 2546, 2563, 2580, 2597, 2614, 2631, 2648, 2665, 2682, 2699, 2716

In [22]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
for n in [1..30]:
    p=prime(n)
    for k in [1..20]: 
        data=[]
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [1..200]:
            m=p^j
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            data=data+[o]
        if data==[]:print("power: ",power," prime: ",p," empty")
        if data!=[]:
            print("----------------------------------------------------------------------------------")
            ul=uniteList(data)
            print("power: ",power," prime: ",p,"first: ",data[0])
            print()
            dt=difference_table(data);print();print(dt)

----------------------------------------------------------------------------------
power:  2  prime:  2 first:  12


[-4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
----------------------------------------------------------------------------------
power:  3  prime:  2 first:  17


[-5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

----------------------------------------------------------------------------------
power:  13  prime:  2 first:  85


[-33, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13]
----------------------------------------------------------------------------------
p

----------------------------------------------------------------------------------
power:  3  prime:  3 first:  0


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
----------------------------------------------------------------------------------
power:  4  prime:  3 first:  1


[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4

----------------------------------------------------------------------------------
power:  14  prime:  3 first:  3


[14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14]
----------------------------------------------------------------------------------
pow

----------------------------------------------------------------------------------
power:  5  prime:  5 first:  3


[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
----------------------------------------------------------------------------------
power:  6  prime:  5 first:  2


[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6

...

----------------------------------------------------------------------------------
power:  19  prime:  107 first:  19


[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]
----------------------------------------------------------------------------------


----------------------------------------------------------------------------------
power:  10  prime:  109 first:  10


[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
----------------------------------------------------------------------------------


----------------------------------------------------------------------------------
power:  19  prime:  109 first:  19


[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]
----------------------------------------------------------------------------------


----------------------------------------------------------------------------------
power:  10  prime:  113 first:  10


[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
----------------------------------------------------------------------------------


----------------------------------------------------------------------------------
power:  19  prime:  113 first:  19


[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]
----------------------------------------------------------------------------------


In [30]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
no2=[]
no3=[]
for n in [1..30]:
    p=prime(n)
    for k in [0..20]: 
        data=[]
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [2..200]:
            m=p^j
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            data=data+[o]
        if data==[]:print("power: ",power," prime: ",p," empty")
        if data!=[]:
            dt=difference_table(data);ul=uniteList(dt)
            print("prime: ",p,"power: ",power,"first: ",data[0],"ul: ",ul)
            if ul[0]!=power:no=no+[(p,power)]
            if len(ul)>1:no2=no2+[(p,power)]
            if len(ul)==0:no3=no3+[(p,power)]
print("============================================================================================")
print("no:");print(no);print();print("no2:");print(no2);print();print("no3:");print(no3)
                    
            

prime:  2 power:  1 first:  4 ul:  [1]
prime:  2 power:  2 first:  8 ul:  [2]
prime:  2 power:  3 first:  12 ul:  [3]
prime:  2 power:  4 first:  16 ul:  [4]
prime:  2 power:  5 first:  20 ul:  [5]
prime:  2 power:  6 first:  24 ul:  [6]
prime:  2 power:  7 first:  28 ul:  [7]
prime:  2 power:  8 first:  32 ul:  [8]
prime:  2 power:  9 first:  36 ul:  [9]
prime:  2 power:  10 first:  40 ul:  [10]
prime:  2 power:  11 first:  44 ul:  [11]
prime:  2 power:  12 first:  48 ul:  [12]
prime:  2 power:  13 first:  52 ul:  [13]
prime:  2 power:  14 first:  56 ul:  [14]
prime:  2 power:  15 first:  60 ul:  [15]
prime:  2 power:  16 first:  64 ul:  [16]
prime:  2 power:  17 first:  68 ul:  [17]
prime:  2 power:  18 first:  72 ul:  [18]
prime:  2 power:  19 first:  76 ul:  [19]
prime:  2 power:  20 first:  80 ul:  [20]
prime:  2 power:  21 first:  84 ul:  [21]
prime:  3 power:  1 first:  1 ul:  [1]
prime:  3 power:  2 first:  3 ul:  [2]
prime:  3 power:  3 first:  3 ul:  [3]
prime:  3 power:  4 f

prime:  29 power:  10 first:  20 ul:  [10]
prime:  29 power:  11 first:  22 ul:  [11]
prime:  29 power:  12 first:  24 ul:  [12]
prime:  29 power:  13 first:  25 ul:  [13]
prime:  29 power:  14 first:  28 ul:  [14]
prime:  29 power:  15 first:  30 ul:  [15]
prime:  29 power:  16 first:  32 ul:  [16]
prime:  29 power:  17 first:  34 ul:  [17]
prime:  29 power:  18 first:  36 ul:  [18]
prime:  29 power:  19 first:  38 ul:  [19]
prime:  29 power:  20 first:  40 ul:  [20]
prime:  29 power:  21 first:  42 ul:  [21]
prime:  31 power:  1 first:  2 ul:  [1]
prime:  31 power:  2 first:  4 ul:  [2]
prime:  31 power:  3 first:  6 ul:  [3]
prime:  31 power:  4 first:  8 ul:  [4]
prime:  31 power:  5 first:  10 ul:  [5]
prime:  31 power:  6 first:  12 ul:  [6]
prime:  31 power:  7 first:  14 ul:  [7]
prime:  31 power:  8 first:  16 ul:  [8]
prime:  31 power:  9 first:  18 ul:  [9]
prime:  31 power:  10 first:  20 ul:  [10]
prime:  31 power:  11 first:  22 ul:  [11]
prime:  31 power:  12 first:  24 

prime:  67 power:  17 first:  34 ul:  [17]
prime:  67 power:  18 first:  36 ul:  [18]
prime:  67 power:  19 first:  38 ul:  [19]
prime:  67 power:  20 first:  40 ul:  [20]
prime:  67 power:  21 first:  42 ul:  [21]
prime:  71 power:  1 first:  2 ul:  [1]
prime:  71 power:  2 first:  4 ul:  [2]
prime:  71 power:  3 first:  6 ul:  [3]
prime:  71 power:  4 first:  8 ul:  [4]
prime:  71 power:  5 first:  11 ul:  [5]
prime:  71 power:  6 first:  12 ul:  [6]
prime:  71 power:  7 first:  14 ul:  [7]
prime:  71 power:  8 first:  16 ul:  [8]
prime:  71 power:  9 first:  18 ul:  [9]
prime:  71 power:  10 first:  20 ul:  [10]
prime:  71 power:  11 first:  22 ul:  [11]
prime:  71 power:  12 first:  24 ul:  [12]
prime:  71 power:  13 first:  26 ul:  [13]
prime:  71 power:  14 first:  28 ul:  [14]
prime:  71 power:  15 first:  30 ul:  [15]
prime:  71 power:  16 first:  31 ul:  [16]
prime:  71 power:  17 first:  34 ul:  [17]
prime:  71 power:  18 first:  36 ul:  [18]
prime:  71 power:  19 first:  38 

prime:  109 power:  2 first:  4 ul:  [2]
prime:  109 power:  3 first:  6 ul:  [3]
prime:  109 power:  4 first:  8 ul:  [4]
prime:  109 power:  5 first:  10 ul:  [5]
prime:  109 power:  6 first:  12 ul:  [6]
prime:  109 power:  7 first:  14 ul:  [7]
prime:  109 power:  8 first:  16 ul:  [8]
prime:  109 power:  9 first:  18 ul:  [9]
prime:  109 power:  10 first:  20 ul:  [10]
prime:  109 power:  11 first:  22 ul:  [11]
prime:  109 power:  12 first:  24 ul:  [12]
prime:  109 power:  13 first:  26 ul:  [13]
prime:  109 power:  14 first:  28 ul:  [14]
prime:  109 power:  15 first:  30 ul:  [15]
prime:  109 power:  16 first:  32 ul:  [16]
prime:  109 power:  17 first:  34 ul:  [17]
prime:  109 power:  18 first:  36 ul:  [18]
prime:  109 power:  19 first:  38 ul:  [19]
prime:  109 power:  20 first:  40 ul:  [20]
prime:  109 power:  21 first:  42 ul:  [21]
prime:  113 power:  1 first:  2 ul:  [1]
prime:  113 power:  2 first:  4 ul:  [2]
prime:  113 power:  3 first:  6 ul:  [3]
prime:  113 powe

In [5]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
no2=[]
no3=[]
for n in [3..3]:
    p=prime(n)
    for k in [16..16]: 
        data=[]
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [2..200]:
            m=p^j
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            data=data+[o]
        if data==[]:print("power: ",power," prime: ",p," empty")
        if data!=[]:
            dt=difference_table(data);ul=uniteList(dt)
            print("prime: ",p,"power: ",power,"first: ",data[0],"ul: ",ul)
            if ul[0]!=power:no=no+[(p,power)]
            if len(ul)>1:no2=no2+[(p,power)]
            if len(ul)==0:no3=no3+[(p,power)]
print("============================================================================================")
print("no:");print(no);print();print("no2:");print(no2);print();print("no3:");print(no3)
print();
print("data:");print(data);print();print("differences:");print(dt)
                    
            

prime:  5 power:  17 first:  29 ul:  [16, 17]
no:
[(5, 17)]

no2:
[(5, 17)]

no3:
[]

data:
[29, 45, 62, 79, 96, 113, 130, 147, 164, 181, 198, 215, 232, 249, 266, 283, 300, 317, 334, 351, 368, 385, 402, 419, 436, 453, 470, 487, 504, 521, 538, 555, 572, 589, 606, 623, 640, 657, 674, 691, 708, 725, 742, 759, 776, 793, 810, 827, 844, 861, 878, 895, 912, 929, 946, 963, 980, 997, 1014, 1031, 1048, 1065, 1082, 1099, 1116, 1133, 1150, 1167, 1184, 1201, 1218, 1235, 1252, 1269, 1286, 1303, 1320, 1337, 1354, 1371, 1388, 1405, 1422, 1439, 1456, 1473, 1490, 1507, 1524, 1541, 1558, 1575, 1592, 1609, 1626, 1643, 1660, 1677, 1694, 1711, 1728, 1745, 1762, 1779, 1796, 1813, 1830, 1847, 1864, 1881, 1898, 1915, 1932, 1949, 1966, 1983, 2000, 2017, 2034, 2051, 2068, 2085, 2102, 2119, 2136, 2153, 2170, 2187, 2204, 2221, 2238, 2255, 2272, 2289, 2306, 2323, 2340, 2357, 2374, 2391, 2408, 2425, 2442, 2459, 2476, 2493, 2510, 2527, 2544, 2561, 2578, 2595, 2612, 2629, 2646, 2663, 2680, 2697, 2714, 2731, 2748, 2765

In [6]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
no2=[]
no3=[]
for n in [3..3]:
    p=prime(n)
    for k in [16..16]: 
        data=[]
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [3..1003]: #nb
            m=p^j
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            data=data+[o]
        if data==[]:print("power: ",power," prime: ",p," empty")
        if data!=[]:
            dt=difference_table(data);ul=uniteList(dt)
            print("prime: ",p,"power: ",power,"first: ",data[0],"ul: ",ul)
            if ul[0]!=power:no=no+[(p,power)]
            if len(ul)>1:no2=no2+[(p,power)]
            if len(ul)==0:no3=no3+[(p,power)]
print("============================================================================================")
print("no:");print(no);print();print("no2:");print(no2);print();print("no3:");print(no3)
print();
print("data:");print(data);print();print("differences:");print(dt)
                    

prime:  5 power:  17 first:  45 ul:  [17]
no:
[]

no2:
[]

no3:
[]

data:
[45, 62, 79, 96, 113, 130, 147, 164, 181, 198, 215, 232, 249, 266, 283, 300, 317, 334, 351, 368, 385, 402, 419, 436, 453, 470, 487, 504, 521, 538, 555, 572, 589, 606, 623, 640, 657, 674, 691, 708, 725, 742, 759, 776, 793, 810, 827, 844, 861, 878, 895, 912, 929, 946, 963, 980, 997, 1014, 1031, 1048, 1065, 1082, 1099, 1116, 1133, 1150, 1167, 1184, 1201, 1218, 1235, 1252, 1269, 1286, 1303, 1320, 1337, 1354, 1371, 1388, 1405, 1422, 1439, 1456, 1473, 1490, 1507, 1524, 1541, 1558, 1575, 1592, 1609, 1626, 1643, 1660, 1677, 1694, 1711, 1728, 1745, 1762, 1779, 1796, 1813, 1830, 1847, 1864, 1881, 1898, 1915, 1932, 1949, 1966, 1983, 2000, 2017, 2034, 2051, 2068, 2085, 2102, 2119, 2136, 2153, 2170, 2187, 2204, 2221, 2238, 2255, 2272, 2289, 2306, 2323, 2340, 2357, 2374, 2391, 2408, 2425, 2442, 2459, 2476, 2493, 2510, 2527, 2544, 2561, 2578, 2595, 2612, 2629, 2646, 2663, 2680, 2697, 2714, 2731, 2748, 2765, 2782, 2799, 2816, 28

In [10]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
no2=[]
no3=[]
for n in [1..30]:
    p=prime(n)
    for k in [0..20]: 
        data=[]
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [3..1003]: # nb
            m=p^j
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            data=data+[o]
        if data==[]:print("power: ",power," prime: ",p," empty")
        if data!=[]:
            dt=difference_table(data);ul=uniteList(dt)
            print("prime: ",p,"power: ",power,"first: ",data[0],"ul: ",ul)
            if ul[0]!=power:no=no+[(p,power)]
            if len(ul)>1:no2=no2+[(p,power)]
            if len(ul)==0:no3=no3+[(p,power)]
print("============================================================================================")
print("no:");print(no);print();print("no2:");print(no2);print();print("no3:");print(no3)

prime:  2 power:  1 first:  5 ul:  [1]
prime:  2 power:  2 first:  10 ul:  [2]
prime:  2 power:  3 first:  15 ul:  [3]
prime:  2 power:  4 first:  20 ul:  [4]
prime:  2 power:  5 first:  25 ul:  [5]
prime:  2 power:  6 first:  30 ul:  [6]
prime:  2 power:  7 first:  35 ul:  [7]
prime:  2 power:  8 first:  40 ul:  [8]
prime:  2 power:  9 first:  45 ul:  [9]
prime:  2 power:  10 first:  50 ul:  [10]
prime:  2 power:  11 first:  55 ul:  [11]
prime:  2 power:  12 first:  60 ul:  [12]
prime:  2 power:  13 first:  65 ul:  [13]
prime:  2 power:  14 first:  70 ul:  [14]
prime:  2 power:  15 first:  75 ul:  [15]
prime:  2 power:  16 first:  80 ul:  [16]
prime:  2 power:  17 first:  85 ul:  [17]
prime:  2 power:  18 first:  90 ul:  [18]
prime:  2 power:  19 first:  95 ul:  [19]
prime:  2 power:  20 first:  100 ul:  [20]
prime:  2 power:  21 first:  105 ul:  [21]
prime:  3 power:  1 first:  2 ul:  [1]
prime:  3 power:  2 first:  5 ul:  [2]
prime:  3 power:  3 first:  6 ul:  [3]
prime:  3 power:  

prime:  29 power:  10 first:  30 ul:  [10]
prime:  29 power:  11 first:  33 ul:  [11]
prime:  29 power:  12 first:  36 ul:  [12]
prime:  29 power:  13 first:  38 ul:  [13]
prime:  29 power:  14 first:  42 ul:  [14]
prime:  29 power:  15 first:  45 ul:  [15]
prime:  29 power:  16 first:  48 ul:  [16]
prime:  29 power:  17 first:  51 ul:  [17]
prime:  29 power:  18 first:  54 ul:  [18]
prime:  29 power:  19 first:  57 ul:  [19]
prime:  29 power:  20 first:  60 ul:  [20]
prime:  29 power:  21 first:  63 ul:  [21]
prime:  31 power:  1 first:  3 ul:  [1]
prime:  31 power:  2 first:  6 ul:  [2]
prime:  31 power:  3 first:  9 ul:  [3]
prime:  31 power:  4 first:  12 ul:  [4]
prime:  31 power:  5 first:  15 ul:  [5]
prime:  31 power:  6 first:  18 ul:  [6]
prime:  31 power:  7 first:  21 ul:  [7]
prime:  31 power:  8 first:  24 ul:  [8]
prime:  31 power:  9 first:  27 ul:  [9]
prime:  31 power:  10 first:  30 ul:  [10]
prime:  31 power:  11 first:  33 ul:  [11]
prime:  31 power:  12 first:  36

prime:  67 power:  16 first:  48 ul:  [16]
prime:  67 power:  17 first:  51 ul:  [17]
prime:  67 power:  18 first:  54 ul:  [18]
prime:  67 power:  19 first:  57 ul:  [19]
prime:  67 power:  20 first:  60 ul:  [20]
prime:  67 power:  21 first:  63 ul:  [21]
prime:  71 power:  1 first:  3 ul:  [1]
prime:  71 power:  2 first:  6 ul:  [2]
prime:  71 power:  3 first:  9 ul:  [3]
prime:  71 power:  4 first:  12 ul:  [4]
prime:  71 power:  5 first:  16 ul:  [5]
prime:  71 power:  6 first:  18 ul:  [6]
prime:  71 power:  7 first:  21 ul:  [7]
prime:  71 power:  8 first:  24 ul:  [8]
prime:  71 power:  9 first:  27 ul:  [9]
prime:  71 power:  10 first:  30 ul:  [10]
prime:  71 power:  11 first:  33 ul:  [11]
prime:  71 power:  12 first:  36 ul:  [12]
prime:  71 power:  13 first:  39 ul:  [13]
prime:  71 power:  14 first:  42 ul:  [14]
prime:  71 power:  15 first:  45 ul:  [15]
prime:  71 power:  16 first:  47 ul:  [16]
prime:  71 power:  17 first:  51 ul:  [17]
prime:  71 power:  18 first:  54

prime:  107 power:  21 first:  63 ul:  [21]
prime:  109 power:  1 first:  3 ul:  [1]
prime:  109 power:  2 first:  6 ul:  [2]
prime:  109 power:  3 first:  9 ul:  [3]
prime:  109 power:  4 first:  12 ul:  [4]
prime:  109 power:  5 first:  15 ul:  [5]
prime:  109 power:  6 first:  18 ul:  [6]
prime:  109 power:  7 first:  21 ul:  [7]
prime:  109 power:  8 first:  24 ul:  [8]
prime:  109 power:  9 first:  27 ul:  [9]
prime:  109 power:  10 first:  30 ul:  [10]
prime:  109 power:  11 first:  33 ul:  [11]
prime:  109 power:  12 first:  36 ul:  [12]
prime:  109 power:  13 first:  39 ul:  [13]
prime:  109 power:  14 first:  42 ul:  [14]
prime:  109 power:  15 first:  45 ul:  [15]
prime:  109 power:  16 first:  48 ul:  [16]
prime:  109 power:  17 first:  51 ul:  [17]
prime:  109 power:  18 first:  54 ul:  [18]
prime:  109 power:  19 first:  57 ul:  [19]
prime:  109 power:  20 first:  60 ul:  [20]
prime:  109 power:  21 first:  63 ul:  [21]
prime:  113 power:  1 first:  3 ul:  [1]
prime:  113 

In [16]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
no2=[]
no3=[]
for n in [5..30]:
    p=prime(n)
    for k in [0..20]: 
        data=[]
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..100]:
            mp=ZZ(mod(m,p))
            dsm=ZZ(digitsum(p,m))
            if dsm==2:
                polydm=evaluate(polyd,m)
                o=order(p,polydm)
                data=data+[o]
        if data==[]:print("prime: ",p,"power: ",power," empty")
        if (data!=[])&(len(data)>1):
            dt=difference_table(data);ul=uniteList(dt)
            print("prime: ",p,"power: ",power,"first: ",data[0],"ul: ",ul)
            if ul[0]!=power:no=no+[(p,power,ul[0])]
            if len(ul)>1:no2=no2+[(p,power)]
            if len(ul)==0:no3=no3+[(p,power)]
print("============================================================================================")
print("no:");print(no);print();print("no2:");print(no2);print();print("no3:");print(no3)

prime:  11 power:  1 first:  0 ul:  [1]
prime:  11 power:  2 first:  0 ul:  [2]
prime:  11 power:  3 first:  0 ul:  [3]
prime:  11 power:  4 first:  0 ul:  [4]
prime:  11 power:  5 first:  0 ul:  [5]
prime:  11 power:  6 first:  0 ul:  [6]
prime:  11 power:  7 first:  0 ul:  [7]
prime:  11 power:  8 first:  0 ul:  [8]
prime:  11 power:  9 first:  0 ul:  [9]
prime:  11 power:  10 first:  0 ul:  [10]
prime:  11 power:  11 first:  0 ul:  [9]
prime:  11 power:  12 first:  1 ul:  [9]
prime:  11 power:  13 first:  -1 ul:  [11]
prime:  11 power:  14 first:  0 ul:  [12]
prime:  11 power:  15 first:  0 ul:  [13]
prime:  11 power:  16 first:  0 ul:  [14]
prime:  11 power:  17 first:  0 ul:  [15]
prime:  11 power:  18 first:  -1 ul:  [16]
prime:  11 power:  19 first:  0 ul:  [17]
prime:  11 power:  20 first:  0 ul:  [18]
prime:  11 power:  21 first:  0 ul:  [19]
prime:  13 power:  1 first:  0 ul:  [1]
prime:  13 power:  2 first:  0 ul:  [2]
prime:  13 power:  3 first:  0 ul:  [3]
prime:  13 power

prime:  43 power:  13 first:  0 ul:  [13]
prime:  43 power:  14 first:  0 ul:  [14]
prime:  43 power:  15 first:  0 ul:  [15]
prime:  43 power:  16 first:  0 ul:  [16]
prime:  43 power:  17 first:  0 ul:  [17]
prime:  43 power:  18 first:  0 ul:  [18]
prime:  43 power:  19 first:  0 ul:  [19]
prime:  43 power:  20 first:  0 ul:  [20]
prime:  43 power:  21 first:  0 ul:  [21]
prime:  47 power:  1 first:  0 ul:  [1]
prime:  47 power:  2 first:  -1 ul:  [2]
prime:  47 power:  3 first:  0 ul:  [3]
prime:  47 power:  4 first:  0 ul:  [4]
prime:  47 power:  5 first:  0 ul:  [5]
prime:  47 power:  6 first:  0 ul:  [6]
prime:  47 power:  7 first:  0 ul:  [7]
prime:  47 power:  8 first:  0 ul:  [8]
prime:  47 power:  9 first:  -1 ul:  [9]
prime:  47 power:  10 first:  0 ul:  [10]
prime:  47 power:  11 first:  1 ul:  [10]
prime:  47 power:  12 first:  0 ul:  [12]
prime:  47 power:  13 first:  0 ul:  [13]
prime:  47 power:  14 first:  0 ul:  [14]
prime:  47 power:  15 first:  0 ul:  [15]
prime:  

In [20]:
prime(15)

47

In [21]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
no2=[]
no3=[]
for n in [16..66]:
    p=prime(n)
    for k in [0..30]: 
        data=[]
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for m in [3..1003]:
            mp=ZZ(mod(m,p))
            dsm=ZZ(digitsum(p,m))
            if dsm==2:
                polydm=evaluate(polyd,m)
                o=order(p,polydm)
                data=data+[o]
        if data==[]:print("prime: ",p,"power: ",power," empty")
        if (data!=[])&(len(data)>1):
            dt=difference_table(data);ul=uniteList(dt)
            print("prime: ",p,"power: ",power,"first: ",data[0],"ul: ",ul)
            if ul[0]!=power:no=no+[(p,power,ul[0])]
            if len(ul)>1:no2=no2+[(p,power)]
            if len(ul)==0:no3=no3+[(p,power)]
print("============================================================================================")
print("no:");print(no);print();print("no2:");print(no2);print();print("no3:");print(no3)

prime:  53 power:  1 first:  0 ul:  [1]
prime:  53 power:  2 first:  0 ul:  [2]
prime:  53 power:  3 first:  0 ul:  [3]
prime:  53 power:  4 first:  0 ul:  [4]
prime:  53 power:  5 first:  0 ul:  [5]
prime:  53 power:  6 first:  0 ul:  [6]
prime:  53 power:  7 first:  0 ul:  [7]
prime:  53 power:  8 first:  0 ul:  [8]
prime:  53 power:  9 first:  0 ul:  [9]
prime:  53 power:  10 first:  0 ul:  [10]
prime:  53 power:  11 first:  0 ul:  [11]
prime:  53 power:  12 first:  0 ul:  [12]
prime:  53 power:  13 first:  0 ul:  [13]
prime:  53 power:  14 first:  0 ul:  [14]
prime:  53 power:  15 first:  0 ul:  [15]
prime:  53 power:  16 first:  0 ul:  [16]
prime:  53 power:  17 first:  0 ul:  [17]
prime:  53 power:  18 first:  0 ul:  [18]
prime:  53 power:  19 first:  0 ul:  [19]
prime:  53 power:  20 first:  0 ul:  [20]
prime:  53 power:  21 first:  0 ul:  [21]
prime:  53 power:  22 first:  0 ul:  [22]
prime:  53 power:  23 first:  0 ul:  [23]
prime:  53 power:  24 first:  0 ul:  [24]
prime:  53

prime:  79 power:  25 first:  0 ul:  [25]
prime:  79 power:  26 first:  0 ul:  [26]
prime:  79 power:  27 first:  0 ul:  [27]
prime:  79 power:  28 first:  0 ul:  [28]
prime:  79 power:  29 first:  0 ul:  [29]
prime:  79 power:  30 first:  0 ul:  [30]
prime:  79 power:  31 first:  0 ul:  [31]
prime:  83 power:  1 first:  0 ul:  [1]
prime:  83 power:  2 first:  0 ul:  [2]
prime:  83 power:  3 first:  0 ul:  [3]
prime:  83 power:  4 first:  0 ul:  [4]
prime:  83 power:  5 first:  0 ul:  [5]
prime:  83 power:  6 first:  0 ul:  [6]
prime:  83 power:  7 first:  0 ul:  [7]
prime:  83 power:  8 first:  0 ul:  [8]
prime:  83 power:  9 first:  0 ul:  [9]
prime:  83 power:  10 first:  0 ul:  [10]
prime:  83 power:  11 first:  0 ul:  [11]
prime:  83 power:  12 first:  0 ul:  [12]
prime:  83 power:  13 first:  0 ul:  [13]
prime:  83 power:  14 first:  0 ul:  [14]
prime:  83 power:  15 first:  0 ul:  [15]
prime:  83 power:  16 first:  0 ul:  [16]
prime:  83 power:  17 first:  0 ul:  [17]
prime:  83

prime:  109 power:  23 first:  0 ul:  [23]
prime:  109 power:  24 first:  0 ul:  [24]
prime:  109 power:  25 first:  0 ul:  [25]
prime:  109 power:  26 first:  0 ul:  [26]
prime:  109 power:  27 first:  0 ul:  [27]
prime:  109 power:  28 first:  0 ul:  [28]
prime:  109 power:  29 first:  0 ul:  [29]
prime:  109 power:  30 first:  0 ul:  [30]
prime:  109 power:  31 first:  0 ul:  [31]
prime:  113 power:  1 first:  0 ul:  [1]
prime:  113 power:  2 first:  0 ul:  [2]
prime:  113 power:  3 first:  0 ul:  [3]
prime:  113 power:  4 first:  0 ul:  [4]
prime:  113 power:  5 first:  0 ul:  [5]
prime:  113 power:  6 first:  0 ul:  [6]
prime:  113 power:  7 first:  0 ul:  [7]
prime:  113 power:  8 first:  0 ul:  [8]
prime:  113 power:  9 first:  0 ul:  [9]
prime:  113 power:  10 first:  0 ul:  [10]
prime:  113 power:  11 first:  0 ul:  [11]
prime:  113 power:  12 first:  0 ul:  [12]
prime:  113 power:  13 first:  0 ul:  [13]
prime:  113 power:  14 first:  0 ul:  [14]
prime:  113 power:  15 first:

prime:  151 power:  5 first:  0 ul:  [5]
prime:  151 power:  6 first:  0 ul:  [6]
prime:  151 power:  7 first:  0 ul:  [7]
prime:  151 power:  8 first:  0 ul:  [8]
prime:  151 power:  9 first:  0 ul:  [9]
prime:  151 power:  10 first:  0 ul:  [10]
prime:  151 power:  11 first:  0 ul:  [11]
prime:  151 power:  12 first:  0 ul:  [12]
prime:  151 power:  13 first:  0 ul:  [13]
prime:  151 power:  14 first:  0 ul:  [14]
prime:  151 power:  15 first:  0 ul:  [15]
prime:  151 power:  16 first:  0 ul:  [16]
prime:  151 power:  17 first:  0 ul:  [17]
prime:  151 power:  18 first:  0 ul:  [18]
prime:  151 power:  19 first:  0 ul:  [19]
prime:  151 power:  20 first:  0 ul:  [20]
prime:  151 power:  21 first:  0 ul:  [21]
prime:  151 power:  22 first:  0 ul:  [22]
prime:  151 power:  23 first:  0 ul:  [23]
prime:  151 power:  24 first:  0 ul:  [24]
prime:  151 power:  25 first:  0 ul:  [25]
prime:  151 power:  26 first:  0 ul:  [26]
prime:  151 power:  27 first:  0 ul:  [27]
prime:  151 power:  2

prime:  181 power:  20 first:  0 ul:  [20]
prime:  181 power:  21 first:  0 ul:  [21]
prime:  181 power:  22 first:  0 ul:  [22]
prime:  181 power:  23 first:  0 ul:  [23]
prime:  181 power:  24 first:  0 ul:  [24]
prime:  181 power:  25 first:  0 ul:  [25]
prime:  181 power:  26 first:  0 ul:  [26]
prime:  181 power:  27 first:  0 ul:  [27]
prime:  181 power:  28 first:  0 ul:  [28]
prime:  181 power:  29 first:  0 ul:  [29]
prime:  181 power:  30 first:  0 ul:  [30]
prime:  181 power:  31 first:  0 ul:  [31]
prime:  191 power:  1 first:  0 ul:  [1]
prime:  191 power:  2 first:  0 ul:  [2]
prime:  191 power:  3 first:  0 ul:  [3]
prime:  191 power:  4 first:  0 ul:  [4]
prime:  191 power:  5 first:  0 ul:  [5]
prime:  191 power:  6 first:  0 ul:  [6]
prime:  191 power:  7 first:  0 ul:  [7]
prime:  191 power:  8 first:  0 ul:  [8]
prime:  191 power:  9 first:  0 ul:  [9]
prime:  191 power:  10 first:  0 ul:  [10]
prime:  191 power:  11 first:  0 ul:  [11]
prime:  191 power:  12 first:

prime:  227 power:  8 first:  0 ul:  [8]
prime:  227 power:  9 first:  0 ul:  [9]
prime:  227 power:  10 first:  0 ul:  [10]
prime:  227 power:  11 first:  0 ul:  [11]
prime:  227 power:  12 first:  0 ul:  [12]
prime:  227 power:  13 first:  0 ul:  [13]
prime:  227 power:  14 first:  0 ul:  [14]
prime:  227 power:  15 first:  0 ul:  [15]
prime:  227 power:  16 first:  0 ul:  [16]
prime:  227 power:  17 first:  0 ul:  [17]
prime:  227 power:  18 first:  0 ul:  [18]
prime:  227 power:  19 first:  0 ul:  [19]
prime:  227 power:  20 first:  0 ul:  [20]
prime:  227 power:  21 first:  0 ul:  [21]
prime:  227 power:  22 first:  0 ul:  [22]
prime:  227 power:  23 first:  0 ul:  [23]
prime:  227 power:  24 first:  0 ul:  [24]
prime:  227 power:  25 first:  0 ul:  [25]
prime:  227 power:  26 first:  0 ul:  [26]
prime:  227 power:  27 first:  0 ul:  [27]
prime:  227 power:  28 first:  0 ul:  [28]
prime:  227 power:  29 first:  0 ul:  [29]
prime:  227 power:  30 first:  0 ul:  [30]
prime:  227 pow

prime:  257 power:  21 first:  0 ul:  [21]
prime:  257 power:  22 first:  0 ul:  [22]
prime:  257 power:  23 first:  0 ul:  [23]
prime:  257 power:  24 first:  0 ul:  [24]
prime:  257 power:  25 first:  0 ul:  [25]
prime:  257 power:  26 first:  0 ul:  [26]
prime:  257 power:  27 first:  0 ul:  [27]
prime:  257 power:  28 first:  0 ul:  [28]
prime:  257 power:  29 first:  0 ul:  [29]
prime:  257 power:  30 first:  0 ul:  [30]
prime:  257 power:  31 first:  0 ul:  [31]
prime:  263 power:  1 first:  0 ul:  [1]
prime:  263 power:  2 first:  0 ul:  [2]
prime:  263 power:  3 first:  0 ul:  [3]
prime:  263 power:  4 first:  0 ul:  [4]
prime:  263 power:  5 first:  0 ul:  [5]
prime:  263 power:  6 first:  0 ul:  [6]
prime:  263 power:  7 first:  0 ul:  [7]
prime:  263 power:  8 first:  0 ul:  [8]
prime:  263 power:  9 first:  0 ul:  [9]
prime:  263 power:  10 first:  0 ul:  [10]
prime:  263 power:  11 first:  0 ul:  [11]
prime:  263 power:  12 first:  0 ul:  [12]
prime:  263 power:  13 first:

prime:  293 power:  1 first:  0 ul:  [1]
prime:  293 power:  2 first:  0 ul:  [2]
prime:  293 power:  3 first:  0 ul:  [3]
prime:  293 power:  4 first:  0 ul:  [4]
prime:  293 power:  5 first:  0 ul:  [5]
prime:  293 power:  6 first:  0 ul:  [6]
prime:  293 power:  7 first:  0 ul:  [7]
prime:  293 power:  8 first:  0 ul:  [8]
prime:  293 power:  9 first:  0 ul:  [9]
prime:  293 power:  10 first:  0 ul:  [10]
prime:  293 power:  11 first:  0 ul:  [11]
prime:  293 power:  12 first:  0 ul:  [12]
prime:  293 power:  13 first:  0 ul:  [13]
prime:  293 power:  14 first:  0 ul:  [14]
prime:  293 power:  15 first:  0 ul:  [15]
prime:  293 power:  16 first:  0 ul:  [16]
prime:  293 power:  17 first:  0 ul:  [17]
prime:  293 power:  18 first:  0 ul:  [18]
prime:  293 power:  19 first:  0 ul:  [19]
prime:  293 power:  20 first:  0 ul:  [20]
prime:  293 power:  21 first:  0 ul:  [21]
prime:  293 power:  22 first:  0 ul:  [22]
prime:  293 power:  23 first:  0 ul:  [23]
prime:  293 power:  24 first:

In [ ]:
import pickle
rfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
no2=[]
no3=[]
firsts=[]
for n in [1..30]:
    p=prime(n)
    for k in [0..20]: 
        data=[]
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [3..1003]: # nb
            m=p^j
            mp=ZZ(mod(m,p))
            mp2=ZZ(mod(m,p^2))
            dsm=digitsum(p,m)
            mpm=ZZ(mod(power,m))
            ex=order(p,m) 
            core=m/p^ex
            cmp=ZZ(mod(core,p))
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            mop=ZZ(mod(m,p))
            data=data+[o]
        if data==[]:print("power: ",power," prime: ",p," empty")
        if data!=[]:
            first=data[0]
            firsts=firsts+[(p,power,first)]
            dt=difference_table(data);ul=uniteList(dt)
            print("prime: ",p,"power: ",power,"first: ",data[0],"ul: ",ul)
            if ul[0]!=power:no=no+[(p,power)]
            if len(ul)>1:no2=no2+[(p,power)]
            if len(ul)==0:no3=no3+[(p,power)]
print("============================================================================================")
print("no:");print(no);print();print("no2:");print(no2);print();print("no3:");print(no3)
print();print("firsts:");el=len(firsts)
for nn in [0..el-1]:
    print(firsts[nn])

prime:  2 power:  1 first:  5 ul:  [1]
prime:  2 power:  2 first:  10 ul:  [2]
prime:  2 power:  3 first:  15 ul:  [3]
prime:  2 power:  4 first:  20 ul:  [4]
prime:  2 power:  5 first:  25 ul:  [5]
prime:  2 power:  6 first:  30 ul:  [6]
prime:  2 power:  7 first:  35 ul:  [7]
prime:  2 power:  8 first:  40 ul:  [8]
prime:  2 power:  9 first:  45 ul:  [9]
prime:  2 power:  10 first:  50 ul:  [10]
prime:  2 power:  11 first:  55 ul:  [11]
prime:  2 power:  12 first:  60 ul:  [12]
prime:  2 power:  13 first:  65 ul:  [13]
prime:  2 power:  14 first:  70 ul:  [14]
prime:  2 power:  15 first:  75 ul:  [15]
prime:  2 power:  16 first:  80 ul:  [16]
prime:  2 power:  17 first:  85 ul:  [17]
prime:  2 power:  18 first:  90 ul:  [18]
prime:  2 power:  19 first:  95 ul:  [19]
prime:  2 power:  20 first:  100 ul:  [20]
prime:  2 power:  21 first:  105 ul:  [21]
prime:  3 power:  1 first:  2 ul:  [1]
prime:  3 power:  2 first:  5 ul:  [2]
prime:  3 power:  3 first:  6 ul:  [3]
prime:  3 power:  

prime:  29 power:  10 first:  30 ul:  [10]
prime:  29 power:  11 first:  33 ul:  [11]
prime:  29 power:  12 first:  36 ul:  [12]
prime:  29 power:  13 first:  38 ul:  [13]
prime:  29 power:  14 first:  42 ul:  [14]
prime:  29 power:  15 first:  45 ul:  [15]
prime:  29 power:  16 first:  48 ul:  [16]
prime:  29 power:  17 first:  51 ul:  [17]
prime:  29 power:  18 first:  54 ul:  [18]
prime:  29 power:  19 first:  57 ul:  [19]
prime:  29 power:  20 first:  60 ul:  [20]
prime:  29 power:  21 first:  63 ul:  [21]
prime:  31 power:  1 first:  3 ul:  [1]
prime:  31 power:  2 first:  6 ul:  [2]
prime:  31 power:  3 first:  9 ul:  [3]
prime:  31 power:  4 first:  12 ul:  [4]
prime:  31 power:  5 first:  15 ul:  [5]
prime:  31 power:  6 first:  18 ul:  [6]
prime:  31 power:  7 first:  21 ul:  [7]
prime:  31 power:  8 first:  24 ul:  [8]
prime:  31 power:  9 first:  27 ul:  [9]
prime:  31 power:  10 first:  30 ul:  [10]
prime:  31 power:  11 first:  33 ul:  [11]
prime:  31 power:  12 first:  36

prime:  67 power:  16 first:  48 ul:  [16]
prime:  67 power:  17 first:  51 ul:  [17]
prime:  67 power:  18 first:  54 ul:  [18]
prime:  67 power:  19 first:  57 ul:  [19]
prime:  67 power:  20 first:  60 ul:  [20]
prime:  67 power:  21 first:  63 ul:  [21]
prime:  71 power:  1 first:  3 ul:  [1]
prime:  71 power:  2 first:  6 ul:  [2]
prime:  71 power:  3 first:  9 ul:  [3]
prime:  71 power:  4 first:  12 ul:  [4]
prime:  71 power:  5 first:  16 ul:  [5]
prime:  71 power:  6 first:  18 ul:  [6]
prime:  71 power:  7 first:  21 ul:  [7]
prime:  71 power:  8 first:  24 ul:  [8]
prime:  71 power:  9 first:  27 ul:  [9]
prime:  71 power:  10 first:  30 ul:  [10]
prime:  71 power:  11 first:  33 ul:  [11]
prime:  71 power:  12 first:  36 ul:  [12]
prime:  71 power:  13 first:  39 ul:  [13]
prime:  71 power:  14 first:  42 ul:  [14]
prime:  71 power:  15 first:  45 ul:  [15]
prime:  71 power:  16 first:  47 ul:  [16]
prime:  71 power:  17 first:  51 ul:  [17]
prime:  71 power:  18 first:  54